# Get PubMed Gene Info

So Pubmed is a nice website which has many databases all working together to fulfill search queries.

The one I care about is given a human gene name --> get the PubMed gene-id --> use the gene_id to get the gene page --> use the gene page to get the sequence information from genbank.

The genbank does not have an api & all their data files can be found through [their ftp server](https://ftp.ncbi.nlm.nih.gov/genbank/release.notes/). However, there is an issue as human and plant and all sorts of organisms are mixed together. Parsing this will be a pain :/

I will opt to be an asshole & use the human apis as at least I can figure out how they work & they get me to the solution.

# Imports and Globals 🌎

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import regex as re

# Make test Request to Gene Cards

In [5]:
selected_gene = "TM6SF2"
genecard_url = 'https://www.genecards.org/cgi-bin/carddisp.pl?gene={}'

selected_url = genecard_url.format(selected_gene)

payload={}

UAS = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1", 
       "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
       "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
       "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
       )

ua = UAS[random.randrange(len(UAS))]
headers = {
  'user-agent':ua
}

response = requests.request("GET", selected_url, headers=headers, data=payload)

#print(response.text)

soup = BeautifulSoup(response.text, 'html.parser')
all_divs = soup.findAll("section",id="summaries")
print(len(all_divs))
#sub_soup = BeautifulSoup(all_divs[0],"html.parser")
all_subsections = all_divs[0].findAll("div",class_="gc-subsection")
print(len(all_subsections))
print(all_subsections[0])

h1 = soup.findAll("h1",id="geneSymbol")
if h1 and h1[0]:
  print(h1[0].findAll("small"))

1
5
<div class="gc-subsection">
<div class="gc-subsection-header">
<h3>Entrez Gene Summary for TM6SF2 Gene</h3>
<a class="gc-ga-link glyphicon glyphicon-new-window" data-ga-action="ENT" data-ga-source-accession="53345" href="https://www.ncbi.nlm.nih.gov/gene/53345#summary" target="_blank" title="See NCBI Entrez Gene entry for TM6SF2"></a>
</div>
<ul class="list-unstyled">
<li>
<p>Enables identical protein binding activity. Involved in regulation of lipid metabolic process. Located in endoplasmic reticulum membrane and endoplasmic reticulum-Golgi intermediate compartment membrane. [provided by Alliance of Genome Resources, Apr 2022]</p>
</li>
</ul>
</div>
[<small>
                    Gene
                        <span id="geneDescription" title="Transmembrane 6 Superfamily Member 2">- Transmembrane 6 Superfamily Member 2</span>
</small>]


ok the requests & bs4 is setup correctly

# Make Test Request to PubMed

## Incremental Dev

In [6]:
response = requests.request("GET", "https://www.ncbi.nlm.nih.gov/gene/?term=TM6SF2", headers=headers, data=payload)

In [11]:
soup = BeautifulSoup(response.text, 'html.parser')
titles = soup.findAll("a",id="feat_gene_title")

In [12]:
titles

[<a data-ga-action="click_feat_title" data-ga-label="feat_gene_title" href="https://www.ncbi.nlm.nih.gov/gene/53345" id="feat_gene_title" ref="discoId=KnownItemSensor&amp;itemType=gene&amp;itemId=gene:53345&amp;linkPos=0">TM6SF2 – transmembrane 6 superfamily member 2</a>]

In [15]:
taxons = soup.findAll("p",class_="ncbi-doc-authors")
taxons

[<p class="ncbi-doc-authors">
 <a data-ga-action="click_feat_taxname" data-ga-label="feat_taxname" href="https://www.ncbi.nlm.nih.gov/labs/data-hub/taxonomy/9606"><span data-item-id="taxonomy:9606" data-item-type="taxon"><i>Homo sapiens</i></span> (human)</a>
 </p>]

In [16]:
titles[0]["href"]

'https://www.ncbi.nlm.nih.gov/gene/53345'

In [19]:
titles[0].contents[0].split()[0]

'TM6SF2'

In [22]:
taxons[0].find("i").contents[0]

'Homo sapiens'

In [ ]:
# https://www.ncbi.nlm.nih.gov/gene/?term=TM6SF2